<a href="https://colab.research.google.com/github/ImOkay-Ms/2020_dacon_ssu.com/blob/master/Baseline_plus_pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pycaret설치
이미 설치되어 있다면 건너뛰자

In [ ]:
!pip install pycaret

     |████████████████████████████████| 256kB 3.5MB/s 
     |████████████████████████████████| 174kB 8.2MB/s 
     |████████████████████████████████| 266kB 9.9MB/s 
     |████████████████████████████████| 1.8MB 16.1MB/s 
     |████████████████████████████████| 13.9MB 18.3MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 6.8MB 65.6MB/s 
     |████████████████████████████████| 1.6MB 64.6MB/s 
     |████████████████████████████████| 266kB 60.7MB/s 
     |████████████████████████████████| 66.1MB 58kB/s 

     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 604kB 46.5MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 3.1MB 58.6MB/s 
     |████████████████████████████████| 153kB 58.3MB/s 
     |████████████████████████████████| 327kB 62.1MB/s 
     |████████████████████████████████| 163kB 67.2MB/s 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

#모델링
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error

# Colab에서 사용하기위한 코드
jupyter notebook에서 실행시 건너뛰자
근데 colab에서 하면 2배이상 빠름

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 경로설정
경로설정해주자

In [ ]:
train=pd.read_csv('/content/drive/My Drive/dacon/심리성향예측/open data/train.csv', index_col=0)
test=pd.read_csv('/content/drive/My Drive/dacon/심리성향예측/open data/test_x.csv', index_col=0)
submission=pd.read_csv('/content/drive/My Drive/dacon/심리성향예측/open data/sample_submission.csv', index_col=0)  
print(train.shape)
print(test.shape)
print(submission.shape)

(45532, 77)
(11383, 76)
(11383, 1)


In [ ]:
train.columns

Index(['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE',
       'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE',
       'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE',
       'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE',
       'age_group', 'education', 'engnat', 'familysize', 'gender', 'hand',
       'married', 'race', 'religion', 'tp01', 'tp02', 'tp03', 'tp04', 'tp05',
       'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'voted', 'wf_01',
       'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06',
       'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13'],
      dtype='object')

# Drop할 변수 설정
이곳이 point

In [ ]:
drop_val = ['QaA', 'QbA', 'QbE', 'QcA', 'QcE', 'QdE', 'QeA','QeE',
       'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE',
       'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE',
       'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE','tp01', 'tp02', 'tp03', 'tp04', 'tp05',
       'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'wf_01',
       'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06',
       'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13']

train = train.drop(drop_val, axis = 1)
test = test.drop(drop_val, axis = 1)
train.head()
test.head()

,QaE,QdA,age_group,education,engnat,familysize,gender,hand,married,race,religion
index,,,,,,,,,,,
0,736,1.0,10s,2,2,3,Male,1,1,White,Christian_Protestant
1,514,3.0,10s,2,2,2,Male,2,1,Other,Christian_Catholic
2,500,2.0,30s,3,2,7,Male,1,2,Other,Christian_Other
3,669,2.0,60s,4,1,4,Male,1,2,White,Christian_Catholic
4,499,2.0,10s,2,1,3,Male,1,1,White,Agnostic


# 실행
이제부터는 수정없이 쭉쭉실행하면댐

In [ ]:
from pycaret.classification import *

In [ ]:
# 'voted' 컬럼이 예측 대상이므로 target 인자에 명시
# 'voted' column is the target variable
#clf = setup(data = train, target = 'voted',ignore_features=['index'],normalize = True)  #index를 무시하고 정규화하는 code인데 이런부분은 나중에 고려하자
clf = setup(data = train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,7302
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 12)"
4,Missing Values,False
5,Numeric Features,3
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [ ]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extreme Gradient Boosting,0.6910,0.7630,0.6603,0.7457,0.7003,0.3839,0.3870,0.3257
1,Gradient Boosting Classifier,0.6907,0.7627,0.6502,0.7511,0.6968,0.3844,0.3888,4.0523
2,Light Gradient Boosting Machine,0.6920,0.7613,0.6429,0.7572,0.6953,0.3880,0.3934,0.3374
3,CatBoost Classifier,0.6912,0.7599,0.6500,0.7518,0.6971,0.3856,0.3899,16.1688
4,Linear Discriminant Analysis,0.6878,0.7576,0.6698,0.7357,0.7012,0.3759,0.3778,0.2279
5,Ada Boost Classifier,0.6867,0.7555,0.6425,0.7490,0.6916,0.3771,0.3818,1.2675
6,Naive Bayes,0.6708,0.7468,0.5233,0.8175,0.6326,0.3588,0.3930,0.0221
7,Quadratic Discriminant Analysis,0.5727,0.7308,0.3338,0.8257,0.4338,0.1828,0.2634,0.0794
8,Logistic Regression,0.6595,0.7294,0.7173,0.6904,0.6968,0.3061,0.3065,0.3540
9,Random Forest Classifier,0.6487,0.7029,0.6232,0.7012,0.6598,0.2992,0.3015,0.1240


In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6822,0.7525,0.6463,0.7397,0.6898,0.3670,0.3705
1,0.6932,0.7653,0.6463,0.7571,0.6973,0.3901,0.3952
2,0.6917,0.7642,0.6477,0.7539,0.6968,0.3869,0.3915
3,0.6923,0.7624,0.6402,0.7594,0.6947,0.3892,0.3949
4,0.7010,0.7716,0.6703,0.7553,0.7102,0.4036,0.4067
Mean,0.6921,0.7632,0.6502,0.7531,0.6978,0.3874,0.3918
SD,0.0060,0.0062,0.0104,0.0069,0.0068,0.0117,0.0118


In [ ]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6906,0.763,0.657,0.7467,0.699,0.3833,0.3866


In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
predictions

,QaE,QdA,age_group,education,engnat,familysize,gender,hand,married,race,religion,Label,Score
0,736,1.0,10s,2,2,3,Male,1,1,White,Christian_Protestant,2,0.6748
1,514,3.0,10s,2,2,2,Male,2,1,Other,Christian_Catholic,2,0.8559
2,500,2.0,30s,3,2,7,Male,1,2,Other,Christian_Other,1,0.4066
3,669,2.0,60s,4,1,4,Male,1,2,White,Christian_Catholic,1,0.2014
4,499,2.0,10s,2,1,3,Male,1,1,White,Agnostic,2,0.7519
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,427,1.0,20s,3,2,3,Female,1,1,Other,Christian_Catholic,1,0.4730
11379,314,1.0,10s,2,2,1,Female,1,1,Asian,Agnostic,2,0.8886
11380,627,2.0,30s,4,1,4,Male,1,2,White,Atheist,1,0.2878
11381,539,1.0,40s,2,1,3,Female,1,2,White,Atheist,1,0.3985


In [ ]:
submission['voted'] = predictions['Score']

# `csv파일생성`

In [ ]:
submission.to_csv('/content/drive/My Drive/dacon/심리성향예측/submission_original.csv', index = True)

#생성파일확인
파일이 잘 생성되었나 read 해와서 확인
index와 voted열으로만 이루어져 있으면 정상

In [ ]:
sub=pd.read_csv('/content/drive/My Drive/dacon/심리성향예측/submission_original.csv', index_col=0)

In [ ]:
sub.head(20)

,voted
index,
0,0.6748
1,0.8559
2,0.4066
3,0.2014
4,0.7519
5,0.5250
6,0.8903
7,0.5230
8,0.3140
